In [3]:
import pandas as pd
import numpy as np
import csv
from google.colab import drive
import matplotlib.pyplot as plt
import time
import statistics

## BEFORE YOU RUN THE CODE, YOU HAVE TO RUN THIS AND FOLLOW THE DIRECTIONS THAT POP UP
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
battery_capacity = 80000.0
degradation_percent = 0.00038
max_cycles = 200
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

In [5]:
def get_data(filename):
  df = pd.read_csv(filename)
  return df

In [68]:
def avg_calc(data):
  ## Create an empty dataframe to add to later
  averages = pd.DataFrame()

  ## Iterate through all of the months
  for month in range(1,data['Month'].max() + 1):
    print(month)

    ## Iterate through all of the days in that month
    for day in range(1,data[data['Month'] == month]['Day'].max() + 1):

      ## Get the data where the month and the day equals the ones that your iterating through
      current_day = data.loc[(data['Month'] == month) & (data['Day'] == day)]
      avgs = pd.DataFrame()
      
      ## Iterate over all of the elements in the currentday dataframe, stopping at hour 21. 
      for index, row in current_day.iterrows():
        if not (row['Hour_Begin'] == 21 or row['Hour_Begin'] == 22 or row['Hour_Begin'] == 23):
            hr1 = float(current_day.loc[index + 0, ]['Hourly_Prices'])
            hr2 = float(current_day.loc[index + 1, ]['Hourly_Prices'])
            hr3 = float(current_day.loc[index + 2, ]['Hourly_Prices'])
            hr4 = float(current_day.loc[index + 3, ]['Hourly_Prices'])

            ## Get the average of the hourly prices in four hour time windows. Get the max and min of each day along with their corresponding hours
            avgs = avgs.append({'hour': current_day.loc[index + 0, ]['Hour_Begin'], 'average_for_next_3_hours':np.mean([hr1,hr2,hr3,hr4])}, ignore_index=True)
      print(avgs[avgs['average_for_next_3_hours'] == avgs['average_for_next_3_hours'].max()]['hour'])
      max_hour = float(avgs[avgs['average_for_next_3_hours'] == avgs['average_for_next_3_hours'].max()]['hour'].min())
      max_val = float(avgs[avgs['hour'] == max_hour]['average_for_next_3_hours'])
      if max_hour < 4:
        max_hour = None
        max_val = None
        min_val = None
        min_hour = None
      else:
        try:
          min_val = float(avgs[avgs['average_for_next_3_hours'] == avgs[avgs['hour'] < float(max_hour)]['average_for_next_3_hours'].min()]['average_for_next_3_hours'])
          min_hour = float(avgs[avgs['average_for_next_3_hours'] == float(min_val)]['hour'])
        except:
          min_hour = float(avgs[avgs['average_for_next_3_hours'] == avgs[avgs['hour'] < float(max_hour)]['average_for_next_3_hours'].min()]['hour'].min())
          min_val = float(avgs[avgs['hour'] == min_hour]['average_for_next_3_hours'])

      averages = averages.append({'year': int(current_day['Year'].iloc[0]), 'month': int(month), 'day': int(day), 'max': float(max_val), 'min': float(min_val), 'hour_max': float(max_hour)
                                  , 'hour_min': float(min_hour), 'difference' : float(max_val - min_val)},  ignore_index=True)
  return averages

In [69]:
data = get_data('/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2016_prices.csv')

In [ ]:
def top_200_calc(averages, data):
  current_charge = 0
  ## Get the top 200 days with the highest max average price and put them in order of month and day
  top210 = averages.sort_values(['difference'], ascending=False).head(210)
  # top210 = averages.sort_values(['max'], ascending=False)
  top210 = top210.sort_values(['month', 'day'])

  ## Set adjusted energy to metered load for all rows (updating adjusted energy for the rows with min and max energy below)
  data['Adjusted_Energy'] = data['Metered_Load']
  adjusted_capacity = battery_capacity

  ## Iterate through the top200 days dataset
  data['is_charging'] = 0
  data['is_discharging'] = 0
  days = 0

  for index, row in top210.iterrows():
    if days < 200:
      discharge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year'])].sort_values(['Hourly_Prices'], ascending=False)[0:4]
      discharge_hours = discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
      last_charge_hour = int(discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True).head(1)['Hour_Begin']) - 1
      charge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] <= last_charge_hour)].sort_values(['Hourly_Prices'], ascending=True)[0:4]
      charge_hours = charge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
      if last_charge_hour >= 3:
        for item in list(charge_hours.index): 
          data.loc[item, 'is_charging'] = 1
          data.loc[item, 'Adjusted_Energy'] += ((current_battery_capacity - 8000)/4)/.98
        for item in list(discharge_hours.index):
          data.loc[item, 'is_discharging'] = 1
          data.loc[item, 'Adjusted_Energy'] -= ((current_battery_capacity - 8000)/4)
        days += 1  
      else:
        continue
      adjusted_capacity = adjusted_capacity * (1 - degradation_percent)

  ## Multiply the new adjusted energy by the hourly prices to get the adjusted hourly cost
  data['Adjusted_Hourly_Cost'] = (data['Hourly_Prices'] * data['Adjusted_Energy'])/1000

  return data, current_battery_capacity

In [ ]:
def top_200_calc_summer_adjusted(averages, data):
  current_charge = 0
  ## Get the top 200 days with the highest max average price and put them in order of month and day
  top210 = averages.sort_values(['difference'], ascending=False).head(210)
  # top210 = averages.sort_values(['max'], ascending=False)
  top210 = top210.sort_values(['month', 'day'])

  ## Set adjusted energy to metered load for all rows (updating adjusted energy for the rows with min and max energy below)
  data['Adjusted_Energy'] = data['Metered_Load']
  adjusted_capacity = battery_capacity

  ## Iterate through the top200 days dataset
  data['is_charging'] = 0
  data['is_discharging'] = 0
  data['is_holding'] = 0
  days = 0
  data['planning_capactiy'] = 0
  count = 0 

  summer_days = 0
  sum_days = []
  winter_days = 0

  for index, row in top210.iterrows():
    if days < 200:
      if ((winter_days < 15) and (row['month'] == 12 or row['month'] == 1 or row['month'] == 2)):
        discharge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] > 7)].sort_values(['Hourly_Prices'], ascending=False)[0:4]
        discharge_hours = discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
        b = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & ((data['Hour_Begin'] == 0) | (data['Hour_Begin'] == 1) | (data['Hour_Begin'] == 2))]
        a = data.loc[data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] == 0)].index - 1]
        charge_hours = pd.concat([a, b])
        max_hour = discharge_hours['Hour_Begin'].min()
        min_hour = 2
        holding_hours = data.loc[(data['Day'] == row['day']) & (data['Year'] == row['year']) & (data['Month'] == row['month']) & (data['Hour_Begin'] > min_hour) & (data['Hour_Begin'] < max_hour)]
        for item in list(charge_hours.index): 
          data.loc[item, 'is_charging'] = 1
          data.loc[item, 'Adjusted_Energy'] += ((current_battery_capacity - 8000)/4)/.98
        for item in list(discharge_hours.index):
          data.loc[item, 'is_discharging'] = 1
          data.loc[item, 'Adjusted_Energy'] -= ((current_battery_capacity - 8000)/4)
        for item in list(holding_hours.index):
          data.loc[item, 'is_holding'] = 1
        days += 1  
        winter_days += 1
        adjusted_capacity = adjusted_capacity * (1 - degradation_percent)
      elif ((summer_days < 15) and (row['month'] == 6 or row['month'] == 7 or row['month'] == 8)):
        charge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] < 15)].sort_values(['Hourly_Prices'], ascending=True)[0:4]
        charge_hours = charge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
        discharge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] > 18)].sort_values(['Hourly_Prices'], ascending=False)[0:4]
        discharge_hours = discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
        max_hour = discharge_hours['Hour_Begin'].min()
        min_hour = charge_hours['Hour_Begin'].max()
        holding_hours = data.loc[(data['Day'] == row['day']) & (data['Year'] == row['year']) & (data['Month'] == row['month']) & (data['Hour_Begin'] > min_hour) & (data['Hour_Begin'] < max_hour)]
        for item in list(charge_hours.index): 
          data.loc[item, 'is_charging'] = 1
          data.loc[item, 'Adjusted_Energy'] += ((current_battery_capacity - 8000)/4)/.98
        for item in list(discharge_hours.index):
          data.loc[item, 'is_discharging'] = 1
          data.loc[item, 'Adjusted_Energy'] -= ((current_battery_capacity - 8000)/4)
        for item in list(holding_hours.index):
          data.loc[item, 'is_holding'] = 1
        days += 1 
        summer_days += 1
        adjusted_capacity = adjusted_capacity * (1 - degradation_percent)
      else:
        discharge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year'])].sort_values(['Hourly_Prices'], ascending=False)[0:4]
        discharge_hours = discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
        last_charge_hour = int(discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True).head(1)['Hour_Begin']) - 1
        charge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] <= last_charge_hour)].sort_values(['Hourly_Prices'], ascending=True)[0:4]
        charge_hours = charge_hours.sort_values(['CT_Hour_Begin'], ascending=True)
        max_hour = discharge_hours['Hour_Begin'].min()
        min_hour = charge_hours['Hour_Begin'].max()
        holding_hours = data.loc[(data['Day'] == row['day']) & (data['Year'] == row['year']) & (data['Month'] == row['month']) & (data['Hour_Begin'] > min_hour) & (data['Hour_Begin'] < max_hour)]
        if last_charge_hour >= 3:
          for item in list(charge_hours.index): 
            data.loc[item, 'is_charging'] = 1
            data.loc[item, 'Adjusted_Energy'] += ((current_battery_capacity - 8000)/4)/.98
          for item in list(discharge_hours.index):
            data.loc[item, 'is_discharging'] = 1
            data.loc[item, 'Adjusted_Energy'] -= ((current_battery_capacity - 8000)/4)
          for item in list(holding_hours.index):
            data.loc[item, 'is_holding'] = 1
          days += 1  
          adjusted_capacity = adjusted_capacity * (1 - degradation_percent)

  ## Multiply the new adjusted energy by the hourly prices to get the adjusted hourly cost
  data['Adjusted_Hourly_Cost'] = (data['Hourly_Prices'] * data['Adjusted_Energy'])/1000

  return data

In [ ]:
def get_running_charge(data,current_battery_capacity, degradation_percent):
  current_state_charge = 8000
  running_sum = 0 
  num_cycles = 0 
  data['running_state_of_charge'] = 0 
  for index, row in data.iterrows():
    if row['is_charging'] == 1:
      running_sum += 1
      current_state_charge += ((current_battery_capacity - 8000)/4)/.98
      data.loc[index, 'running_state_of_charge'] = round(current_state_charge, 2)
    elif row['is_discharging'] == 1:
      running_sum += 1
      current_state_charge -= ((current_battery_capacity - 8000)/4)/.98
      data.loc[index, 'running_state_of_charge'] = round(current_state_charge, 2)
    elif running_sum == 8: 
      running_sum = 0
      num_cycles += 1 
      current_battery_capacity = battery_capacity * (1 - degradation_percent)**num_cycles
      #print(current_battery_capacity)
    else:
      data.loc[index, 'running_state_of_charge'] = round(current_state_charge, 2)
  return data

In [ ]:
def original_adjusted(data):
  difference = data["Original_Hourly_Cost"].sum() - data["Adjusted_Hourly_Cost"].sum()
  original_adjusted_dict = {"Original": data["Original_Hourly_Cost"].sum(), "Adjusted": data["Adjusted_Hourly_Cost"].sum(), 
                            "Difference": difference, "% Difference": 100 * (difference / data["Original_Hourly_Cost"].sum())}

  return original_adjusted_dict

In [ ]:
def sopo(data, year):
  so_data = pd.read_csv('/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/SoPo/SoPo_{}.csv'.format(year))
  joined = pd.merge(so_data, data, on=['Month', 'Day', 'Hour_Begin'], how='left')
  #print(joined[['Month','Day','Hour_Begin','is_discharging']])
  percent_hours_discharged = np.mean(joined['is_discharging']) * 100
  joined['adjusted_difference'] = joined['Metered_Load'] - joined['Adjusted_Energy']
  adjusted_average = np.mean(joined['Metered_Load']) - np.mean(joined['Adjusted_Energy'])
  return adjusted_average, percent_hours_discharged

In [ ]:
def gtc(data, year):
  gtc_data = pd.read_csv('/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/gtc/gtc_{}.csv'.format(year))
  joined = pd.merge(gtc_data, data, on=['Month', 'Day', 'Hour_Begin'], how='left')
  #print(joined[['Month','Day','Hour_Begin','is_discharging']])
  percent_hours_discharged = np.mean(joined['is_discharging']) * 100
  joined['adjusted_difference'] = joined['Metered_Load'] - joined['Adjusted_Energy']
  adjusted_average = np.mean(joined['Metered_Load']) - np.mean(joined['Adjusted_Energy'])
  return adjusted_average, percent_hours_discharged

In [ ]:
#files = ["/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2016_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2017_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2018_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2019_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2020_prices.csv"]
#different test cases
files = ["/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2016_prices.csv","/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/Increased Prices Test/2016_all.csv","/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/Increased Prices Test/2016_outliers.csv","/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/Increased Prices Test/2016_nonoutliers.csv"]
start_time = time.time()
distribution_dict = {"January": [], "February": [], "March": [], "April": [], "May": [], "June": [], "July": [], "August": [], "September": [], "October": [], "November": [], "December": []}
current_battery_capacity = battery_capacity
current_state_of_charge = 8000

# loops through all the files
year = 2016
for file in files:
  time.time() - start_time
  #year = file[119:123]

  data = get_data(file)
  data['running_state_of_charge'] = 0
  #creates a column for the original price of energy
  data['Original_Hourly_Cost'] = data['Metered_Load'] * data['Hourly_Prices']/1000
  # call the avg_calc function
  averages = avg_calc(data)
  print(distribution_dict)
  # call the top_200_calc function
  data = top_200_calc_summer_adjusted(averages, data)
  get_running_charge(data, battery_capacity, degradation_percent)
  print(distribution_dict)
  #calculate sopo_top10
  adjusted_average, percent_hours_discharged = sopo(data,year)
  print("SoPo")
  print("Adjusted Average:",adjusted_average)
  print("Percentage of Hours Discharged:",percent_hours_discharged,"%")

  #calculate gtc_top5
  adjusted_average, percent_hours_discharged = gtc(data,year)
  print("GTC")
  print("Adjusted Average:",adjusted_average)
  print("Percentage of Hours Discharged:",percent_hours_discharged,"%")

  #Uncomment the line next line to output the data as a csv file
  #data.to_csv("/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/top200_yearly/{}.csv".format(year))
  
  # call the original_adjust function
  original_adjusted_dict = original_adjusted(data)
  print(original_adjusted_dict)

  # call the differenct graph functions
  adjusted_days_graph(data, year)
  monthly_graphs(data, year)
  #year = year + 1
  break
  
# get the total run time
total_time = time.time() - start_time
print("Total Run Time: ", total_time)

#data.to_csv()

1
2
3
4
5
6
7
8
9
10
11
12
{'January': [], 'February': [], 'March': [], 'April': [], 'May': [], 'June': [], 'July': [], 'August': [], 'September': [], 'October': [], 'November': [], 'December': []}
{'January': [], 'February': [], 'March': [], 'April': [], 'May': [], 'June': [], 'July': [], 'August': [], 'September': [], 'October': [], 'November': [], 'December': []}
SoPo
Adjusted Average: 10800.0
Percentage of Hours Discharged: 60.0 %
GTC
Adjusted Average: 0.0
Percentage of Hours Discharged: 0.0 %
{'Original': 111199662.76254, 'Adjusted': 110942058.97315225, 'Difference': 257603.78938774765, '% Difference': 0.23165878653593094}


NameError: ignored

In [ ]:
distribution_dict

{'April': [],
 'August': [],
 'December': [],
 'February': [],
 'January': [],
 'July': [],
 'June': [],
 'March': [],
 'May': [],
 'November': [],
 'October': [],
 'September': []}

In [ ]:
data.loc[(data['Month'] == 12) & (data['Day'] == 5)]

,CT_Hour_Begin,Month,Day,Year,Hour_Begin,Metered_Load,Deficit_Energy,Surplus_Energy,Total_Energy_Bought,Hourly_Prices,Hourly_Cost,running_state_of_charge,Original_Hourly_Cost,Adjusted_Energy,is_charging,is_discharging,is_holding,planning_capactiy,Adjusted_Hourly_Cost
8136,12/05/2016 00:00,12,5,2016,0,310543.2,146084.4,0.0,146084.4,22.20,3.242708e+06,26367.35,6894.059040,328910.546939,1,0,0,0,7301.814142
8137,12/05/2016 01:00,12,5,2016,1,303150.1,140229.1,0.0,140229.1,21.82,3.060051e+06,44734.69,6614.735182,321517.446939,1,0,0,0,7015.510692
8138,12/05/2016 02:00,12,5,2016,2,303037.9,135203.2,0.0,135203.2,20.99,2.837428e+06,63102.04,6360.765521,321405.246939,1,0,0,0,6746.296133
8139,12/05/2016 03:00,12,5,2016,3,311036.0,98777.8,0.0,98777.8,21.68,2.141611e+06,81469.39,6743.260480,329403.346939,1,0,0,0,7141.464562
8140,12/05/2016 04:00,12,5,2016,4,341944.9,123259.4,0.0,123259.4,22.99,2.833130e+06,81469.39,7861.313251,341944.900000,0,0,1,0,7861.313251
8141,12/05/2016 05:00,12,5,2016,5,401033.6,169143.7,0.0,169143.7,26.16,4.424935e+06,81469.39,10491.038976,401033.600000,0,0,1,0,10491.038976
8142,12/05/2016 06:00,12,5,2016,6,429586.3,133761.1,0.0,133761.1,27.86,3.727133e+06,81469.39,11968.274318,429586.300000,0,0,1,0,11968.274318
8143,12/05/2016 07:00,12,5,2016,7,423091.9,131063.9,0.0,131063.9,27.21,3.565895e+06,81469.39,11512.330599,423091.900000,0,0,1,0,11512.330599
8144,12/05/2016 08:00,12,5,2016,8,418134.2,116155.6,0.0,116155.6,27.81,3.230299e+06,81469.39,11628.312102,418134.200000,0,0,1,0,11628.312102
8145,12/05/2016 09:00,12,5,2016,9,414156.6,136134.9,0.0,136134.9,28.38,3.863944e+06,81469.39,11753.764308,414156.600000,0,0,1,0,11753.764308


In [ ]:
### SMOOTHED ALGORITHM ATTEMPT #1 ###
### run things above before this! It takes in distribution_dict ###
# print(distribution_dict)

def get_avg_distribution(distribution_dict, max_cycles):
  # std_dev_dict = {k: statistics.stdev(i) for k,i in distribution_dict.items()}
  distribution_avg_dict = {k: sum(i)/len(i) for k,i in distribution_dict.items()}
  
  total = 0
  for i in distribution_avg_dict.items():
    total += i[1]

  rounded_distribution_avg_dict = {k: round(sum(i)/len(i)) for k,i in distribution_dict.items()}
  
  total2 = 0
  for i in rounded_distribution_avg_dict.items():
    total2 += i[1]

  # print(distribution_avg_dict)

  if total2 < max_cycles:
    difference = max_cycles - total2
    while difference > 0:
      max_key = max(distribution_avg_dict, key=distribution_avg_dict.get)
      # print("MAX KEY: ", max_key)
      distribution_avg_dict[max_key] += 1
      rounded_distribution_avg_dict[max_key] += 1
      difference = difference - 1
      total2 = 0
      for i in rounded_distribution_avg_dict.items():
        total2 += i[1]
  ### RACHEL, I ADDED THE STUFF BELOW THE MORNING OF NOVEMBER 16th! ###
  elif total2 > max_cycles:
    difference = total2 - max_cycles
    while difference > 0:
      min_key = min(disttribution_avg_dict, key=distribution_avg_dict.get)
      distribution_avg_dict[min_key] -= 1
      rounded_distribution_avg_dict[min_key] -= 1
      difference = difference - 1
      total2 = 0
      for i in rounded_distribution_avg_dict.itetms():
        total2 += i[1]
  ######################################################################


  

  print("distribution_avg_dict: ", distribution_avg_dict)
  print("total: ", total)
  print("rounded_distribution_avg_dict: ", rounded_distribution_avg_dict)
  print("total2: ", total2)  

  return distribution_avg_dict, total, rounded_distribution_avg_dict, total2




In [ ]:
def smoothed_top_200_calc(averages, data, rounded_distribution_avg_dict):
  months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

  ## IN PROGRESS ##

  ## Set adjusted energy to metered load for all rows (updating adjusted energy for the rows with min and max energy below)
  data['Adjusted_Energy'] = data['Metered_Load']
  adjusted_capacity = battery_capacity

  ## Iterate through the top200 days dataset
  data['is_charging'] = 0
  data['is_discharging'] = 0

  for month in months:
    days = 0
    num_days = rounded_distribution_avg_dict[month]
    buffer = 3

    ## Get the top days for the desired month with the highest max average price and put them in order of day
    topdays = averages.loc[(averages['month'] == months.index(month) + 1)].sort_values(['max'], ascending=False).head(num_days + buffer)
    topdays = topdays.sort_values(['day'])


    for index, row in topdays.iterrows():
      if days < num_days:
        discharge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year'])].sort_values(['Hourly_Prices'], ascending=False)[0:4]
        last_charge_hour = int(discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True).head(1)['Hour_Begin']) - 1
        if last_charge_hour >= 3:
          for item in list(discharge_hours.index):
            data.loc[item, 'is_discharging'] = 1
            data.loc[item , 'Adjusted_Energy'] -= ((adjusted_capacity - 8000)/4)
          charge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] <= last_charge_hour)].sort_values(['Hourly_Prices'], ascending=True)[0:4]
          for item in list(charge_hours.index): 
            data.loc[item, 'is_charging'] = 1
            data.loc[item, 'Adjusted_Energy'] += ((adjusted_capacity - 8000)/4)/.95
          days += 1  
        else:
          continue
          
      adjusted_capacity = adjusted_capacity * (1 - degradation_percent)

    ## Multiply the new adjusted energy by the hourly prices to get the adjusted hourly cost
    data['Adjusted_Hourly_Cost'] = (data['Hourly_Prices'] * data['Adjusted_Energy'])/1000

  return data

In [ ]:
get_avg_distribution(distribution_dict, 200)

ZeroDivisionError: ignored

In [ ]:
files = ["/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2016_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2017_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2018_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2019_prices.csv", "/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2020_prices.csv"]
start_time = time.time()

# loops through all the files
for file in files:
  time.time() - start_time
  year = file[119:123]
  # data = get_data("/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2016_prices.csv")
  
  # call the get_data function
  data = get_data(file)
  
  #creates a column for the original price of energy
  data['Original_Hourly_Cost'] = data['Metered_Load'] * data['Hourly_Prices']/1000

  # call the avg_calc function
  averages = avg_calc(data)

  rounded_distribution_avg_dict = get_avg_distribution(distribution_dict, 200)[2]

  # call the top_200_calc function
  data = smoothed_top_200_calc(averages, data, rounded_distribution_avg_dict)

  #Uncomment the line next line to output the data as a csv file
  #data.to_csv("/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/top200_yearly/{}.csv".format(year))
  
  # call the original_adjust function
  original_adjusted_dict = original_adjusted(data)
  print(original_adjusted_dict)

  # call the differenct graph functions
  smoothed_adjusted_days_graph(data, year)
  monthly_graphs(averages, data, year)
  break

# get the total run time
total_time = time.time() - start_time
print("Total Run Time: ", total_time)

#data.to_csv()

1
2
3
4
5
6
7
8
9
10
11
12


ZeroDivisionError: ignored

In [ ]:
# def get_avg_distribution2(distribution_dict):
#   smoothed_dict = {'January': {}, 'February': {}, 'March': {}, 'April': {}, 'May': {}, 'June': {}, 'July': {}, 'August': {}, 'September': {}, 'October': {}, 'November': {}, 'December': {}}
#   print(distribution_dict)
#   for k in distribution_dict.keys():
#     print(k)
#     smoothed_dict[k]['Average'] = sum(distribution_dict[k])/len(distribution_dict[k])
#     smoothed_dict[k]['Rounded_Average'] = round(distribution_dict[k])/len(distribution_dict[k])
#     smoothed_dict[k]['Standard_Deviation'] = statistics.stdev(distribution_dict[k])

  
#   std_dev_dict = {k: statistics.stdev(i) for k,i in distribution_dict.items()}

#   distribution_avg_dict = {k: sum(i)/len(i) for k,i in distribution_dict.items()}
  
#   total = 0
#   for i in distribution_avg_dict.items():
#     total += i[1]

#   rounded_distribution_avg_dict = {k: round(sum(i)/len(i)) for k,i in distribution_dict.items()}
  
#   total2 = 0
#   for i in rounded_distribution_avg_dict.items():
#     total2 += i[1]

#   # print("distribution_avg_dict: ", distribution_avg_dict)
#   # print("total: ", total)
#   # print("rounded_distribution_avg_dict: ", rounded_distribution_avg_dict)
#   # print("total2: ", total2)  

#   return distribution_avg_dict, total, rounded_distribution_avg_dict, total2, std_dev_dict


In [ ]:
get_avg_distribution2(distribution_dict)


# print(distribution_dict)
#### confused why this isn't working :(

# total2

In [ ]:
adjusted_days_graph(averages)

In [ ]:
monthly_graphs(averages, data)

In [ ]:
data = get_data("/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/2016_prices.csv")
## Create an empty dataframe to add to later
averages = pd.DataFrame()

## Iterate through all of the months
for month in range(1,data['Month'].max() + 1):
  print(month)

  ## Iterate through all of the days in that month
  for day in range(1,data[data['Month'] == month]['Day'].max() + 1):

    ## Get the data where the month and the day equals the ones that your iterating through
    current_day = data.loc[(data['Month'] == month) & (data['Day'] == day)]
    avgs = pd.DataFrame()
    
    ## Iterate over all of the elements in the currentday dataframe, stopping at hour 21. 
    for index, row in current_day.iterrows():
      if not (row['Hour_Begin'] == 21 or row['Hour_Begin'] == 22 or row['Hour_Begin'] == 23):
          hr1 = float(current_day.loc[index + 0, ]['Hourly_Prices'])
          hr2 = float(current_day.loc[index + 1, ]['Hourly_Prices'])
          hr3 = float(current_day.loc[index + 2, ]['Hourly_Prices'])
          hr4 = float(current_day.loc[index + 3, ]['Hourly_Prices'])

          ## Get the average of the hourly prices in four hour time windows. Get the max and min of each day along with their corresponding hours
          avgs = avgs.append({'hour': current_day.loc[index + 0, ]['Hour_Begin'], 'average_for_next_3_hours':np.mean([hr1,hr2,hr3,hr4])}, ignore_index=True)
    averages = averages.append({'year': data['Year'][0], 'month': int(month), 'day': int(day), 'max': avgs['average_for_next_3_hours'].max(), 'min': avgs['average_for_next_3_hours'].min(), 'hour_max':
                                  avgs.loc[avgs['average_for_next_3_hours'] == avgs['average_for_next_3_hours'].max(), ['hour']].iloc[0].hour, 'hour_min':
                                  avgs.loc[avgs['average_for_next_3_hours'] == avgs['average_for_next_3_hours'].min(), ['hour']].iloc[0].hour},  ignore_index=True)

In [ ]:
## Get the top 200 days with the highest max average price and put them in order of month and day
top210 = averages.sort_values(['max'], ascending=False).head(210)
top210 = top210.sort_values(['month', 'day'])
top210

In [ ]:
data['is_charging'] = 0
data['is_discharging'] = 0
days = 0
for index, row in top210.iterrows():
  if days < 200:
    discharge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year'])].sort_values(['Hourly_Prices'], ascending=False)[0:4]
    last_charge_hour = int(discharge_hours.sort_values(['CT_Hour_Begin'], ascending=True).head(1)['Hour_Begin']) - 1
    if last_charge_hour >= 3:
      for item in list(discharge_hours.index):
        data.loc[item, 'is_discharging'] = 1
        data.loc[item , 'Adjusted_Energy'] -= ((adjusted_capacity - 8000)/4)
      charge_hours = data.loc[(data['Day'] == row['day']) & (data['Month'] == row['month']) & (data['Year'] == row['year']) & (data['Hour_Begin'] <= last_charge_hour)].sort_values(['Hourly_Prices'], ascending=True)[0:4]
      for item in list(charge_hours.index): 
        data.loc[item, 'is_charging'] = 1
        data.loc[item, 'Adjusted_Energy'] += ((adjusted_capacity - 8000)/4)/.95
      days += 1  
    else:
      continue

In [ ]:
data.loc[(data['Day'] == 5) & (data['Month'] ==1)]

In [ ]:
## Set adjusted energy to metered load for all rows (updating adjusted energy for the rows with min and max energy below)
data['Adjusted_Energy'] = data['Metered_Load']
adjusted_capacity = battery_capacity

## Iterate through the top200 days dataset
for i in range(len(top200)):
  ## take 20,000 in energy off of the max hour, and the preceding 3 hours. adding 20,000 in energy to the min hours. 
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_max'])), 'Adjusted_Energy'] -= ((adjusted_capacity - 8000)/4)
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_max']) + 1), 'Adjusted_Energy'] -= ((adjusted_capacity - 8000)/4)
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_max']) + 2), 'Adjusted_Energy'] -= ((adjusted_capacity - 8000)/4)
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_max']) + 3), 'Adjusted_Energy'] -= ((adjusted_capacity - 8000)/4)
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_min'])), 'Adjusted_Energy'] += ((adjusted_capacity - 8000)/4)/.95
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_min']) + 1), 'Adjusted_Energy'] += ((adjusted_capacity - 8000)/4)/.95
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_min']) + 2), 'Adjusted_Energy'] += ((adjusted_capacity - 8000)/4)/.95
   data.loc[(data['Month'] == int(top200.iloc[[i]]['month'])) & (data['Day'] == int(top200.iloc[[i]]['day'])) & (data['Hour_Begin'] == int(top200.iloc[[i]]['hour_min']) + 3), 'Adjusted_Energy'] += ((adjusted_capacity - 8000)/4)/.95
   adjusted_capacity = adjusted_capacity * (1 - degradation_percent)

## Multiply the new adjusted energy by the hourly prices to get the adjusted hourly cost
data['Adjusted_Hourly_Cost'] = (data['Hourly_Prices'] * data['Adjusted_Energy'])/1000

In [ ]:
time.time() - start_time

In [ ]:
original_total_cost = data["Original_Hourly_Cost"].sum()
print("Original: ",original_total_cost)

adjusted_total_cost = data["Adjusted_Hourly_Cost"].sum()
print("Adjusted: ",adjusted_total_cost)

difference = original_total_cost-adjusted_total_cost
print("Difference: ",difference)

percent_diff = 100*((original_total_cost-adjusted_total_cost)/original_total_cost)
print("% Difference: ", percent_diff)

In [ ]:
top200 = averages.sort_values(['max'], ascending=False).head(200)
top200.reset_index(drop=True, inplace=True)
top200 = top200.groupby('month').agg('count')

ax = plt.gca()

top200.plot(label='Number of Adjusted Days per Month',kind='bar',y='day',color='c', ax=ax)
plt.show()


In [ ]:
# reset the index of the top200 dataframe
top200 = averages.sort_values(['max'], ascending=False).head(200)
top200.reset_index(drop=True, inplace=True)

# loop through the months of the year
for i in range(12):
  month = data.loc[data['Month'] == i+1] # get desired month
  days = month.groupby(["Month","Day"]).sum() # group by to get the sum of the rows of a specific month and day
  
  top_month = top200.loc[top200['month'] == i+1]
  top_days = top_month.groupby(["month", "day"]).sum()
  num_adjusted_days = len(top_days)

  # plot the data
  ax = plt.gca()
  days.plot(kind='bar',y='Original_Hourly_Cost',color='red', ax=ax)
  days.plot(kind='bar',y='Adjusted_Hourly_Cost',color='turquoise', ax=ax)
  plt.show()

  monthly_original_total_cost = days["Original_Hourly_Cost"].sum()
  print("Original: ",monthly_original_total_cost)

  monthly_adjusted_total_cost = days["Adjusted_Hourly_Cost"].sum()
  print("Adjusted: ",monthly_adjusted_total_cost)

  monthly_difference = monthly_original_total_cost-monthly_adjusted_total_cost
  print("Difference: ",monthly_difference)

  percent_diff = 100*((original_total_cost-adjusted_total_cost)/original_total_cost)
  print("Month % Difference: ", percent_diff) 

  percent_contribution = 100*(1-((difference-monthly_difference)/difference))
  print("% Contribution to Year: ", percent_contribution) 
  
  print("Number of Adjusted Days: ", num_adjusted_days)
  print()


In [ ]:
ss_data = get_data("/content/gdrive/My Drive/ISyE 4106: Senior Design/Problem Formulation/Preliminary Coding & Data Draft/yearly_price_csv/solar_smoothing/SolarSmoothingData.csv")

In [ ]:
bucket = 0 
ss_data['adjusted_2016'] = 0
battery = 0 

for index, row in ss_data.iterrows():
  if row['2016'] > 0:
    bucket += row['2016']
  elif row['2016'] < 0:
    if bucket >= row['2016']:
      bucket -= row['2016']
    elif bucket < row['2016']:
      bucket = 0 
      ss_data['adjusted_2016'] += bucket 
